In [1]:
import numpy as np
import torch

import sys
sys.path.insert(0, '../ml')
from customlayers import *

In [2]:
%load_ext autoreload
%autoreload 2

In [41]:
hparams = {'accelerator': 'gpu',
           'devices': [0],
           'dtype': torch.float64,
           'precision': 64,
           'loss_terms': [F.mse_loss],
           'loss_coeffs': [1],
           'optimizer': torch.optim.Adam, 
           'learning_rate': 1e-4,
           'batch_size': 100,
           'max_epochs': 5000,
           'early_stopping_patience': 1000000,
           'DeepONet': False,
           'VarMiON': True,
           'NGO': False,
           'Q': 12,
           'Q_L': 268,
           'h': 64, 
           'Petrov-Galerkin': False,
           'bias_NLBranch': False,
           'bias_LBranch': False,
           'NLB_outputactivation': nn.Tanhshrink(),
           'Cholesky': False,
           'scale_invariance': False,
           'norm_basis': True,
           'bound_mus': False,
           '1/theta': False,
           'symgroupavg': False,
           'NOMAD': False}

params = {}
params['hparams'] = {}
params['hparams'] = hparams

In [313]:
class InvCNN(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.channels = 1
        
        # Adjusted convolutional layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h'],self.hparams['h'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=self.channels, out_channels=self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=self.channels, out_channels=self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=self.channels, out_channels=1, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.channels,int(self.hparams['h']/16),int(self.hparams['h']/16))))
        self.layers.append(nn.ReLU())
        self.layers.append(InversionLayer())
        self.layers.append(ReshapeLayer(output_shape=(self.channels, int(self.hparams['h']/16),int(self.hparams['h']/16))))
        self.layers.append(nn.ConvTranspose2d(self.channels, self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(self.channels, self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(self.channels, self.channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(self.channels, 1, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['h'],self.hparams['h'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        y = x
        return y

In [314]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [315]:
model = InvCNN(params)

In [316]:
sum(p.numel() for p in model.parameters())

40

In [317]:
model.forward(x)

tensor([[[ 3.0361e-07, -1.4789e-06,  3.4459e-08,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.4026e-06,  1.2247e-06, -1.5879e-07,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -5.8208e-11,
           2.7940e-09, -1.4203e-08],
         ...,
         [-1.4026e-06,  1.2247e-06, -1.5879e-07,  ...,  1.6065e-08,
          -2.0955e-09,  1.8626e-09],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 3.0361e-07, -1.4789e-06,  3.4459e-08,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.4026e-06,  1.2247e-06, -1.5879e-07,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -5.8208e-11,
           2.7940e-09, -1.4203e-08],
         ...,
         [-1.4026e-06,  1